# Visual SLAM

1. Initialization: At first frame, initialize map with 3D points from stereo.
2. Tracking:
   - at frame i+1, match keypoints between i and i+1


In [ ]:
import os
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

from lightglue import LightGlue, SuperPoint
from lightglue.utils import rbd

from lac.perception.vision import LightGlueMatcher
from lac.perception.depth import project_pixel_to_rover
from lac.utils.frames import apply_transform
from lac.utils.plotting import plot_3d_points, plot_surface, plot_poses
from lac.util import load_data, grayscale_to_3ch_tensor
import lac.params as params

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path(
    os.path.expanduser("~/LunarAutonomyChallenge/output/NavAgent/map1_preset4_gtnav_steer")
)
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Num poses: {len(poses)}")

map = np.load("../../data/heightmaps/competition/Moon_Map_01_preset_0.dat", allow_pickle=True)

In [ ]:
# Load the images

left_imgs = {}
right_imgs = {}

for img_name in os.listdir(data_path / "FrontLeft"):
    left_imgs[int(img_name.split(".")[0])] = cv2.imread(
        str(data_path / "FrontLeft" / img_name), cv2.IMREAD_GRAYSCALE
    )

for img_name in os.listdir(data_path / "FrontRight"):
    right_imgs[int(img_name.split(".")[0])] = cv2.imread(
        str(data_path / "FrontRight" / img_name), cv2.IMREAD_GRAYSCALE
    )

assert len(left_imgs.keys()) == len(right_imgs.keys())
img_idxs = sorted(left_imgs.keys())

In [ ]:
fig = plot_surface(map)
fig = plot_poses(poses[::20], fig=fig)
fig.show()

In [ ]:
extractor = SuperPoint(max_num_keypoints=2048).eval().cuda()  # load the extractor
matcher = LightGlue(features="superpoint").eval().cuda()  # load the matcher

## Stereo (PnP) VO


In [ ]:
from lac.localization.slam.visual_odometry import StereoVisualOdometry

In [ ]:
svo = StereoVisualOdometry(cam_config)

In [ ]:
start_idx = 80

svo.initialize(poses[start_idx], left_imgs[start_idx], right_imgs[start_idx])

In [ ]:
svo_poses = [poses[start_idx]]

idx = start_idx

for i in tqdm(range(1980)):
    idx += 2
    svo.track(left_imgs[idx], right_imgs[idx])
    svo_poses.append(svo.rover_pose)

In [ ]:
fig = plot_poses(poses, no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, no_axes=True, fig=fig, color="red", name="Stereo (PnP) VO")
fig.show()

In [ ]:
fig.write_html("stereo_pnp_vo.html")

In [ ]:
idx = 80

feats0_left = extractor.extract(grayscale_to_3ch_tensor(left_imgs[idx]).cuda())
feats0_right = extractor.extract(grayscale_to_3ch_tensor(right_imgs[idx]).cuda())

matches0_stereo = matcher({"image0": feats0_left, "image1": feats0_right})
matches0_stereo = rbd(matches0_stereo)["matches"]  # indices with shape (K,2)
points0_left = rbd(feats0_left)["keypoints"][
    matches0_stereo[..., 0]
]  # coordinates in image #0, shape (K,2)
points0_right = rbd(feats0_right)["keypoints"][
    matches0_stereo[..., 1]
]  # coordinates in image #1, shape (K,2)

disparities = (points0_left - points0_right)[:, 0]
depths = params.FL_X * params.STEREO_BASELINE / disparities

points0_left = points0_left.cpu().numpy()
depths = depths.cpu().numpy()

In [ ]:
points0_rover = []
for pixel, depth in zip(points0_left, depths):
    point_rover = project_pixel_to_rover(pixel, depth, "FrontLeft", cam_config)
    points0_rover.append(point_rover)
points0_rover = np.array(points0_rover)
points0_world = apply_transform(poses[idx], points0_rover)

In [ ]:
feats1_left = extractor.extract(grayscale_to_3ch_tensor(left_imgs[idx + 2]).cuda())

matches01_left = matcher({"image0": feats0_left, "image1": feats1_left})
matches01_left = rbd(matches01_left)["matches"]

In [ ]:
# Extract indices from the first column (img0_left indices)
stereo_indices = matches0_stereo[:, 0]
frame_indices = matches01_left[:, 0]

# Find the intersection of indices
common_indices = torch.tensor(
    list(set(stereo_indices.cpu().numpy()) & set(frame_indices.cpu().numpy()))
).cuda()

# Get the matches from both tensors corresponding to the common indices
stereo_common = matches0_stereo[torch.isin(stereo_indices, common_indices)]
frame_common = matches01_left[torch.isin(frame_indices, common_indices)]
points0_world_common = points0_world[torch.isin(stereo_indices, common_indices).cpu().numpy()]

print(f"Number of common matches: {len(common_indices)}")
# print("Stereo Matches:", stereo_common)
# print("Frame-to-Frame Matches:", frame_common)

In [ ]:
points3D = points0_world_common
points2D = rbd(feats1_left)["keypoints"][frame_common[:, 1]].cpu().numpy()
print(points3D.shape, points2D.shape)

In [ ]:
from lac.utils.frames import invert_transform_mat, OPENCV_TO_CAMERA_PASSIVE, get_cam_pose_rover

In [ ]:
# Camera intrinsics (K)
K = params.CAMERA_INTRINSICS

# Estimate relative motion with PnP (RANSAC for robustness)
success, rvec, tvec, inliers = cv2.solvePnPRansac(
    objectPoints=points3D,
    imagePoints=points2D,
    cameraMatrix=K,
    distCoeffs=None,
    flags=cv2.SOLVEPNP_ITERATIVE,
    reprojectionError=8.0,
    iterationsCount=100,
)

if success:
    # Convert rotation vector to rotation matrix
    R, _ = cv2.Rodrigues(rvec)
    T = np.hstack((R, tvec))  # [R | t]
    est_pose = np.vstack((T, [0, 0, 0, 1]))  # Homogeneous transform (4x4)
    w_T_c = invert_transform_mat(est_pose)  # world to opencv passive
    w_T_c[:3, :3] = w_T_c[:3, :3] @ OPENCV_TO_CAMERA_PASSIVE  # world to camera passive
    rover_to_cam = get_cam_pose_rover("FrontLeft")
    cam_to_rover = invert_transform_mat(rover_to_cam)
    rover_pose = w_T_c @ cam_to_rover

    print("Estimated pose at frame 82:")
    print(rover_pose)
else:
    print("PnP failed to estimate motion.")

In [ ]:
matcher = LightGlueMatcher()

feats0, feats1, matches01 = matcher.match(left_imgs[2], right_imgs[2])
matches = matches01["matches"]  # indices with shape (K,2)
points0 = feats0["keypoints"][matches[..., 0]]  # coordinates in image #0, shape (K,2)
points1 = feats1["keypoints"][matches[..., 1]]  # coordinates in image #1, shape (K,2)

disparities = (points0 - points1)[:, 0]
depths = params.FL_X * params.STEREO_BASELINE / disparities

In [ ]:
def stereo_to_rover_points(left_img, right_img):
    feats0, feats1, matches01 = matcher.match(left_img, right_img)
    matches = matches01["matches"]  # indices with shape (K,2)
    points0 = feats0["keypoints"][matches[..., 0]]  # coordinates in image #0, shape (K,2)
    points1 = feats1["keypoints"][matches[..., 1]]  # coordinates in image #1, shape (K,2)
    descriptors0 = feats0["descriptors"][matches[..., 0]]
    descriptors1 = feats1["descriptors"][matches[..., 1]]

    disparities = (points0 - points1)[:, 0]
    depths = params.FL_X * params.STEREO_BASELINE / disparities

    points_rover = []
    for pixel, depth in zip(points0, depths):
        point_rover = project_pixel_to_rover(pixel, depth, "FrontLeft", cam_config)
        points_rover.append(point_rover)
    return np.array(points_rover), descriptors0

In [ ]:
init_points, descriptors = stereo_to_rover_points(left_imgs[80], right_imgs[80])

In [ ]:
descriptors.shape

In [ ]:
init_points.shape

In [ ]:
all_points_world = []

for i in np.arange(100, 2000, 100):
    points_rover = stereo_to_rover_points(left_imgs[i], right_imgs[i])
    points_world = apply_transform(poses[i], points_rover)
    all_points_world.append(points_world)

all_points_world = np.concatenate(all_points_world, axis=0)

In [ ]:
fig = plot_surface(map)
# fig = plot_poses(poses[::20], fig=fig)
fig = plot_3d_points(all_points_world, fig=fig)
fig.show()

In [ ]:
from dataclasses import dataclass


@dataclass
class MapPoint:
    xyz: np.ndarray
    descriptor: np.ndarray
    label: str

In [ ]:
class SLAM:
    def __init__(self):
        self.matcher = LightGlueMatcher()

In [ ]:
import gtsam
from gtsam.symbol_shorthand import B, V, X, L

In [ ]:
g = 1.622
IMU_PARAMS = gtsam.PreintegrationParams.MakeSharedU(g)
I = np.eye(3)
IMU_PARAMS.setAccelerometerCovariance(I * 0.2)
IMU_PARAMS.setGyroscopeCovariance(I * 0.2)
IMU_PARAMS.setIntegrationCovariance(I * 0.2)

In [ ]:
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

pose_key = X(0)
pose_noise = gtsam.noiseModel.Diagonal.Sigmas(0.01 * np.ones(6))
pose_0 = gtsam.Pose3(poses[0])
graph.push_back(gtsam.PriorFactorPose3(pose_key, pose_0, pose_noise))
initial_estimate.insert(pose_key, pose_0)

velocity_key = V(0)
velocity_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)
velocity_0 = np.zeros(3)
graph.push_back(gtsam.PriorFactorVector(velocity_key, velocity_0, velocity_noise))

# Preintegrator
accum = gtsam.PreintegratedImuMeasurements(IMU_PARAMS)

n_frames = 100

for i in range(0, n_frames):
    accum.integrateMeasurement(imu_data[i, :3], imu_data[i, 3:], params.DT)
    pose_key += 1
    DELTA = gtsam.Pose3(
        gtsam.Rot3.Rodrigues(0, 0, 0.1 * np.random.randn()),
        gtsam.Point3(4 * np.random.randn(), 4 * np.random.randn(), 4 * np.random.randn()),
    )